In [2]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from openai import OpenAI
from dotenv import load_dotenv
import os
import sqlite3
import pandas as pd

In [3]:
openai = OpenAI(
    api_key= "-TjFCsia_r7.oivr6in~4ZJ", # Refer to Create a secret key section
    base_url="https://cloud.olakrutrim.com/v1",
)

In [4]:
db_conn = sqlite3.connect("chinook.db")
result = pd.read_sql_query("SELECT type, sql FROM sqlite_master WHERE sql is not null", db_conn)

In [5]:
db = FAISS.from_texts(result['sql'].to_list(), OllamaEmbeddings(model="nomic-embed-text"))

In [13]:
question = "How many customers are from brazil"
embedder = OllamaEmbeddings(model="nomic-embed-text")
q = embedder.embed_query(question)
a = db.similarity_search_by_vector(q, k=2)

In [18]:
def combineDocs(docs):
    context = "\n\n".join(f'{doc.page_content}' for doc in docs)
    return context

In [22]:
info = combineDocs(a)
info = ""

In [23]:
chat_completion = openai.chat.completions.create(
    model="Meta-Llama-3-8B-Instruct",
    messages=[
    {"role": "system", "content": f"""'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. {info} ===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n3. If the provided context is insufficient, please explain why it can\'t be generated. \n4. Please use the most relevant table(s). \n5. If the question has been asked and answered before, please repeat the answer exactly as it was given before. \n6. Ensure that the output SQL is SQLite-compliant and executable, and free of syntax errors. \n'"""},
    {"role": "user", "content": question}
    ],
    logit_bias= {2435: -100, 640: -100},
    max_tokens= 2000,
    temperature= 0, # Optional, Defaults to 1. Range: 0 to 2
    top_p= 1 # Optional, Defaults to 1. It is generally recommended to alter this or temperature but not both.
)

response = chat_completion.choices[0].message.content

In [21]:
print(info)

CREATE INDEX [IFK_CustomerSupportRepId] ON "customers" ([SupportRepId])

CREATE TABLE "customers"
(
    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)
